In [87]:
import pandas as pd
from transformers import AutoTokenizer

# Evaluation Results

After fine-tuning the models, we need to evaluate their performance. We use [GERBIL](https://gerbil-qa.aksw.org/gerbil/config) to calculate the F1 score of each model and language individually. We convert the questions in the test dataset to SPARQL queries, send them to [Wikidata Query Service](https://query.wikidata.org/) to get answers, and build a dataset in QALD format containing the infered SPARQL queries and answers. Then, we upload this file to GERBIL and evaluate it.

We conduct ablation studies using different models with different tokenizers. We fine-tune the models on the [QALD-9-plus](https://github.com/KGQA/QALD_9_plus) training dataset, a multilingual dataset that extends QALD-9 with more questions in nine languages and covers both DBpedia and Wikidata as knowledge graphs, and evaluate them on the test dataset. Since there are only european languages in the QALD-9-plus dataset, we also add Chinese and Japanese translations of questions to expand the coverage of languages for KGQA research and applications and enable multilingual KGQA models to learn from more diverse and rich data. As the QALD-9-Plus dataset incorporated Spanish questions only recently, some of our experiments might have excluded this language from both training and evaluation.

Note that unless otherwise stated, all answers is limited to 50 results to avoid excessive file size. 

## Experiment 1

The first experiment aims to leverage a multilingual text-to-text transformer model called [mT5-base](https://huggingface.co/google/mt5-base) to answer natural language questions over linked data. 
The model is previously  pre-trained on a large-scale dataset called [LC-QuAD](https://github.com/AskNowQA/LC-QuAD) that contains over 5000 questions and their corresponding SPARQL queries. 
We also filter out entities and relations in LC-QuAD dataset and add them to mt5 tokenizer to improve tokenizing knowledge graph entities. This new tokenizer is trained on LC-QuAD training dataset. 

We fine-tuned a pre-trained language model on different datasets that contain varying numbers of languages with four settings: one-shot, few-shot, no-en, and all languages. 
- In the one-shot setting, we fine-tuned the model on only English questions. 
- In the few-shot setting, we fine-tuned the model on a small amount of languages. 
- In the no-en setting, we exclude English questions from dataset while including all other languages.
- In the all languages setting, we fine-tuned the model on all the available languages.

The experiment runs for 300 epochs before evaluation.

### mT5 tokenizer + LC-QuAD tokens

First, we show you the results of the mT5 tokenizer on tokenizing SPARQL query. 
Note that these SPARQL queries are preprocessed to simplify tokenization and reduce syntax errors.

We take three example SPARQL queries. 

- Which instruments does Cat Stevens play?

In [88]:
query1 = "SELECT DISTINCT var_uri WHERE bra_open wd_Q154216 wdt_P1303 var_uri sep_dot bra_close"

- Is horse racing a sport?

In [89]:
query2 = "ASK WHERE bra_open wd_Q187916 wdt_P279* wd_Q349 sep_dot bra_close"

- Which states border Illinois?

In [90]:
query3 = "SELECT DISTINCT var_uri WHERE bra_open wd_Q1204 wdt_P47 var_uri sep_dot bra_close"

First, we show how mT5 tokenizer tokenizes SPARQL queries.

In [91]:
mt5_tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")

/upb/users/m/mengshim/profiles/unix/cs/miniconda3/envs/mengshi/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [92]:
tokens = mt5_tokenizer.tokenize(query1)
print(tokens, end=", ")

['▁', 'SELECT', '▁D', 'ISTI', 'NCT', '▁var', '_', 'uri', '▁W', 'HERE', '▁bra', '_', 'open', '▁w', 'd', '_', 'Q', '1542', '16', '▁w', 'd', 't', '_', 'P', '1303', '▁var', '_', 'uri', '▁sep', '_', 'dot', '▁bra', '_', 'close'], 

In [93]:
tokens = mt5_tokenizer.tokenize(query2)
print(tokens, end=", ")

['▁', 'ASK', '▁W', 'HERE', '▁bra', '_', 'open', '▁w', 'd', '_', 'Q', '1879', '16', '▁w', 'd', 't', '_', 'P', '279', '*', '▁w', 'd', '_', 'Q', '349', '▁sep', '_', 'dot', '▁bra', '_', 'close'], 

In [94]:
tokens = mt5_tokenizer.tokenize(query3)
print(tokens, end=", ")

['▁', 'SELECT', '▁D', 'ISTI', 'NCT', '▁var', '_', 'uri', '▁W', 'HERE', '▁bra', '_', 'open', '▁w', 'd', '_', 'Q', '1204', '▁w', 'd', 't', '_', 'P', '47', '▁var', '_', 'uri', '▁sep', '_', 'dot', '▁bra', '_', 'close'], 

We see that entities and relations are tokenized as some single letters and numbers, therefore, the meaning of entities and relations is lost. 
We filter out entities and relations from LC-QuAD dataset to expand vocabulary of mT5 tokenizer. 

Now, we demonstrate the tokenization results of our new trained tokenizer. 

In [95]:
lcquad_tokenizer = AutoTokenizer.from_pretrained("../lcquad_tokenizer")

In [96]:
tokens = lcquad_tokenizer.tokenize(query1)
print(tokens, end=", ")

['▁', 'SELECT', '▁D', 'ISTI', 'NCT', '▁var', '_', 'uri', '▁W', 'HERE', '▁', 'bra_open', '▁', 'wd_Q154', '▁216', '▁', 'wdt_P1303', '▁var', '_', 'uri', '▁', 'sep_dot', '▁', 'bra_close'], 

In [97]:
tokens = lcquad_tokenizer.tokenize(query2)
print(tokens, end=", ")

['▁', 'ASK', '▁W', 'HERE', '▁', 'bra_open', '▁', 'wd_Q18', '▁7', '916', '▁', 'wdt_P279', '▁*', '▁', 'wd_Q349', '▁', 'sep_dot', '▁', 'bra_close'], 

In [98]:
tokens = lcquad_tokenizer.tokenize(query3)
print(tokens, end=", ")

['▁', 'SELECT', '▁D', 'ISTI', 'NCT', '▁var', '_', 'uri', '▁W', 'HERE', '▁', 'bra_open', '▁', 'wd_Q1204', '▁', 'wdt_P47', '▁var', '_', 'uri', '▁', 'sep_dot', '▁', 'bra_close'], 

In this case, some entities and relations have been correctly tokenized. 
However, despite the inclusion of many tokens from LC-QuAD into the mT5 tokenizer, it still lacks the complete set of wikidata entities and relations. Consequently, some entities and relations remain incorrectly tokenized. 
The impact of adding entities and relations into a tokenizer on the tokenization quality of SPARQL queries for smaller knowledge graphs is yet to be determined.

### one-shot


We first fine-tune with only English questions, which takes least time for fine-tuning. 

result: https://gerbil-qa.aksw.org/gerbil/experiment?id=202304030010

In [99]:
exp1_one_shot = pd.read_csv("../pred_files/pretrain_lcqald/1/results.csv")
exp1_one_shot

,Language,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,Macro F1 QALD
0,ba,0.0000,0.0000,0.0000,0.0441,0.0441,0.0441,0.0844
1,be,0.0070,0.3323,0.0036,0.0923,0.0953,0.0907,0.1632
2,de,0.0055,0.1401,0.0028,0.0819,0.0821,0.0822,0.1475
3,en,0.0094,0.2216,0.0048,0.1748,0.1785,0.1777,0.2891
4,fr,0.0067,0.7955,0.0034,0.0776,0.0806,0.0760,0.1409
5,hy,0.0031,0.2526,0.0015,0.0411,0.0438,0.0399,0.0767
6,ja,0.0004,0.0216,0.0002,0.0700,0.0701,0.0699,0.1293
7,lt,0.0066,0.2178,0.0034,0.0776,0.0806,0.0760,0.1402
8,ru,0.0039,0.1148,0.0020,0.0793,0.0817,0.0846,0.1510
9,uk,0.0081,0.2075,0.0041,0.0666,0.0746,0.0639,0.1175


The results of this experiment show that English has the best performance among the languages tested. The other languages, such as `ba` and `hy`, have much lower scores. This suggests that the model is more suited for English than for other languages, and that more data and fine-tuning are needed to improve its performance on other languages.

### four-shot 

We experiment with four languages of questions: en, de, zh, and ru.

result: https://gerbil-qa.aksw.org/gerbil/experiment?id=202304250003

In [100]:
exp1_four_shot = pd.read_csv("../pred_files/pretrain_lcqald/4/results.csv")
exp1_four_shot

,Language,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,Macro F1 QALD
0,ba,0.0040,0.3316,0.0020,0.0897,0.0938,0.0887,0.1607
1,be,0.0086,0.1741,0.0044,0.1072,0.1137,0.1055,0.1839
2,de,0.0088,0.1932,0.0045,0.1299,0.1363,0.1281,0.2173
3,en,0.0089,0.2220,0.0045,0.1518,0.1582,0.1500,0.2502
4,fr,0.0067,0.5866,0.0034,0.0775,0.0804,0.0761,0.1412
5,hy,0.0034,0.3354,0.0017,0.0484,0.0510,0.0472,0.0901
6,ja,0.0088,0.1944,0.0045,0.1146,0.1213,0.1129,0.1961
7,lt,0.0091,0.1929,0.0047,0.0971,0.1097,0.0971,0.1729
8,ru,0.0094,0.2104,0.0048,0.1416,0.1468,0.1425,0.2387
9,uk,0.0086,0.1532,0.0044,0.1150,0.1215,0.1132,0.1944


The experiment also reveals that the languages that are included in the training dataset have better performance than the languages that are not. This indicates that the model can learn from the multilingual data and generalize to similar languages. However, the F1 score of English is lower than the one-shot baseline, which means that the model does not benefit from the additional data for other languages and may suffer from interference from other languages.

### all languages (11)

In all languages setting, we include en, de, zh, ja, ru, fr, hy, ba, be, uk, lt. 
`es` is not included since our Spanish translation of questions is merged later than this experiment.

result: https://gerbil-qa.aksw.org/gerbil/experiment?id=202304250002

In [101]:
exp1_all = pd.read_csv("../pred_files/pretrain_lcqald/11/results.csv")
exp1_all

,Language,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,Macro F1 QALD
0,ba,0.0043,0.2334,0.0021,0.1191,0.1232,0.1181,0.2056
1,be,0.0088,0.1972,0.0045,0.1226,0.1291,0.1207,0.2070
2,de,0.0088,0.1898,0.0045,0.1298,0.1363,0.1280,0.2173
3,en,0.0087,0.1673,0.0045,0.1371,0.1435,0.1353,0.2290
4,fr,0.0067,0.5412,0.0034,0.0775,0.0804,0.0761,0.1411
5,hy,0.0034,0.3776,0.0017,0.0631,0.0657,0.0619,0.1165
6,ja,0.0089,0.2129,0.0046,0.1444,0.1510,0.1426,0.2367
7,lt,0.0092,0.1929,0.0047,0.1165,0.1228,0.1148,0.1980
8,ru,0.0087,0.2315,0.0044,0.1224,0.1288,0.1206,0.2082
9,uk,0.0086,0.1823,0.0044,0.1224,0.1288,0.1206,0.2067


The results show that German has the best performance among all the languages. On the other hand, French and Armenian have the worst performance. The other languages do not show any obvious difference in their performance.

In [102]:
# TODO: add bar chart to compare between different settings in exp 1

### no-en

We exclude English question to test the influence on English performance. 

result: https://gerbil-qa.aksw.org/gerbil/experiment?id=202304250002

In [103]:
exp1_no_en = pd.read_csv("../pred_files/pretrain_lcqald/no_en/results.csv")
exp1_no_en

,Language,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,Macro F1 QALD
0,ba,0.0043,0.2334,0.0021,0.1191,0.1232,0.1181,0.2056
1,be,0.0088,0.1972,0.0045,0.1226,0.1291,0.1207,0.2070
2,de,0.0088,0.1898,0.0045,0.1298,0.1363,0.1280,0.2173
3,en,0.0087,0.1673,0.0045,0.1371,0.1435,0.1353,0.2290
4,fr,0.0067,0.5412,0.0034,0.0775,0.0804,0.0761,0.1411
5,hy,0.0034,0.3776,0.0017,0.0631,0.0657,0.0619,0.1165
6,ja,0.0089,0.2129,0.0046,0.1444,0.1510,0.1426,0.2367
7,lt,0.0092,0.1929,0.0047,0.1165,0.1228,0.1148,0.1980
8,ru,0.0087,0.2315,0.0044,0.1224,0.1288,0.1206,0.2082
9,uk,0.0086,0.1823,0.0044,0.1224,0.1288,0.1206,0.2067


The results show that almost all performance is worse than with English questions, indicating that the cross-lingual transfer is not very effective. The only exception is `ja`, which is improved slightly. However, this improvement is not significant when compared to the performance without English questions, suggesting that the English questions do not provide much benefit for the Japanese language.

## Experiment 2

In the second experiment, we try google/mt5-base model with mT5 tokenizer without any pretraining on LC-QuAD dataset to observe whether pretraining on LC-QuAD and adding tokens to tokenizer improve the performance. 

Furthermore, the experiment is run for 300 epochs prior to evaluation.

### one-shot

First, we try with only English questions.

results: https://gerbil-qa.aksw.org/gerbil/experiment?id=202304050000

In [104]:
exp2_one_shot = pd.read_csv("../pred_files/mt5/1_mt5/results.csv")
exp2_one_shot

,Language,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,Macro F1 QALD
0,ba,0.0001,0.0039,0.0000,0.0515,0.0515,0.0515,0.0971
1,be,0.0046,0.1299,0.0023,0.0860,0.0934,0.0854,0.1524
2,de,0.0088,0.1519,0.0046,0.1152,0.1289,0.1112,0.1890
3,en,0.0098,0.2193,0.0050,0.2185,0.2324,0.2143,0.3207
4,fr,0.0064,0.6061,0.0032,0.0650,0.0732,0.0625,0.1175
5,hy,0.0031,0.8727,0.0015,0.0485,0.0512,0.0473,0.0902
6,ja,0.0032,0.0741,0.0016,0.0554,0.0657,0.0521,0.0963
7,lt,0.0063,0.1477,0.0032,0.0503,0.0585,0.0480,0.0903
8,ru,0.0049,0.0936,0.0025,0.1007,0.1086,0.1001,0.1724
9,uk,0.0046,0.0941,0.0023,0.0567,0.0645,0.0621,0.1139


Conclusion:

- `en` has a higher F1 Score than in experiment 1 one-shot setting
- the performance is worse for other languages except for `de` and `ru`

### four-shot

We also choose `en`, `de`, `zh`, and `ru` for four-shot training.

results: https://gerbil-qa.aksw.org/gerbil/experiment?id=202304040000

In [105]:
exp2_four_shot = pd.read_csv("../pred_files/mt5/4_mt5/results.csv")
exp2_four_shot

,Language,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,Macro F1 QALD
0,ba,0.0045,0.2023,0.0023,0.1267,0.1342,0.1256,0.2116
1,be,0.0096,0.1835,0.0049,0.1489,0.1593,0.1514,0.2270
2,de,0.0098,0.1708,0.0051,0.2062,0.2177,0.2033,0.2821
3,en,0.0096,0.1340,0.0050,0.1915,0.2030,0.1886,0.2596
4,fr,0.0068,0.5408,0.0034,0.0775,0.0804,0.0761,0.1409
5,hy,0.0037,0.3118,0.0019,0.0579,0.0597,0.0595,0.1119
6,ja,0.0082,0.1904,0.0042,0.1430,0.1526,0.1396,0.2144
7,lt,0.0088,0.1956,0.0045,0.1340,0.1428,0.1311,0.2110
8,ru,0.0095,0.1313,0.0049,0.1548,0.1664,0.1518,0.2236
9,uk,0.0094,0.1421,0.0048,0.1664,0.1766,0.1661,0.2391


Conculsion:
- `en` F1 score decreases slightly
- F1 Score improves significantly for languages in training dataset.
- Also for `ja`, which has connection to `zh`, and `ba`, `be`, and `uk`, which have connection to `ru`, the F1 Score increases significantly.

### all languages (11)

We also fine-tuned for all 11 languages. 

results: https://gerbil-qa.aksw.org/gerbil/experiment?id=202304080000

In [106]:
exp2_all = pd.read_csv("../pred_files/mt5/11_mt5/results.csv")
exp2_all

,Language,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,Macro F1 QALD
0,ba,0.0056,0.1589,0.0029,0.1571,0.1628,0.1557,0.2457
1,be,0.0098,0.1442,0.0051,0.1986,0.2171,0.1956,0.2704
2,de,0.0099,0.1477,0.0051,0.1962,0.2137,0.1920,0.2634
3,en,0.0099,0.1439,0.0051,0.1914,0.2134,0.1884,0.2585
4,es,0.0100,0.1481,0.0052,0.2059,0.2137,0.2035,0.2787
5,fr,0.0068,0.5354,0.0034,0.0775,0.0804,0.0761,0.1408
6,hy,0.0034,0.1957,0.0017,0.0558,0.0584,0.0546,0.1030
7,ja,0.0104,0.1583,0.0054,0.1966,0.2052,0.1945,0.2625
8,lt,0.0104,0.1307,0.0054,0.1900,0.2089,0.1854,0.2641
9,ru,0.0104,0.1369,0.0054,0.1775,0.1907,0.1742,0.2393


In [107]:
exp1_all

,Language,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,Macro F1 QALD
0,ba,0.0043,0.2334,0.0021,0.1191,0.1232,0.1181,0.2056
1,be,0.0088,0.1972,0.0045,0.1226,0.1291,0.1207,0.2070
2,de,0.0088,0.1898,0.0045,0.1298,0.1363,0.1280,0.2173
3,en,0.0087,0.1673,0.0045,0.1371,0.1435,0.1353,0.2290
4,fr,0.0067,0.5412,0.0034,0.0775,0.0804,0.0761,0.1411
5,hy,0.0034,0.3776,0.0017,0.0631,0.0657,0.0619,0.1165
6,ja,0.0089,0.2129,0.0046,0.1444,0.1510,0.1426,0.2367
7,lt,0.0092,0.1929,0.0047,0.1165,0.1228,0.1148,0.1980
8,ru,0.0087,0.2315,0.0044,0.1224,0.1288,0.1206,0.2082
9,uk,0.0086,0.1823,0.0044,0.1224,0.1288,0.1206,0.2067


To know how the F1 Score decreases with limitation of 50 results, we generate a qald file without limitation. Fortunatly, these files are not very large. 

results: https://gerbil-qa.aksw.org/gerbil/experiment?id=202304080001

In [108]:
exp2_all_wo_limit = pd.read_csv("../pred_files/mt5/11_mt5_all/results.csv")
exp2_all_wo_limit

,Language,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,Macro F1 QALD
0,ba,0.0047,0.0116,0.0030,0.1573,0.1625,0.1563,0.2457
1,be,0.0071,0.0064,0.0080,0.2164,0.2316,0.2151,0.2852
2,de,0.0079,0.0077,0.0080,0.2139,0.2280,0.2115,0.2828
3,en,0.0072,0.0066,0.0081,0.2093,0.2280,0.2083,0.2749
4,es,0.0066,0.0055,0.0082,0.2237,0.2283,0.2234,0.2950
5,fr,0.0111,0.4137,0.0056,0.0880,0.0877,0.0882,0.1613
6,hy,0.0060,0.0144,0.0038,0.0660,0.0659,0.0662,0.1235
7,ja,0.0069,0.0059,0.0083,0.2144,0.2198,0.2140,0.2761
8,lt,0.0076,0.0071,0.0083,0.1931,0.2087,0.1902,0.2650
9,ru,0.0070,0.0060,0.0083,0.1953,0.2051,0.1938,0.2553


The macro F1 Score is improved about 0 to 2 depending on language without limitation.

In Experiment 2 all languages setting, we observed a significant improvement in the performance of our model for all languages. Nine of the twelve languages reached a F1 score above 0.19. However, French and Hungarian still performed worst among the languages, suggesting that they might require more data or a different approach. The results of Experiment 2 clearly surpassed those of Experiment 1, demonstrating the effectiveness of this method.

The results of Experiment 1 and 2 indicate that the one-shot setting achieves the highest F1 Score for `en`. This suggests that multilingual fine-tuning has a negative impact on the performance of a single language. However, in a multilingual KGQA system, the performance of other languages is also relevant. As the number of languages in the training dataset increases, so does the average F1 Score.

In [109]:
# TODO: compare exp1_all and exp2_all in bar chart

The F1 scores for all languages except `hy` are higher than all languages setting in Experiment 1. For `en`, the F1 Score is increased by 5.41.

## Experiment 3

In this experiment, we want to figure out whether the mT5 tokenizer with LC-QuAD tokens really improves the performance. Hence, we use the mT5-base model without pretraining on LC-QuAD from Experiment 2 and tokenizer from Experiment 1. 

The experiment is run for 300 epochs prior to evaluation.

### one-shot

We first try with the one-shot setting, fine-tune with only English questions. 

results: https://gerbil-qa.aksw.org/gerbil/experiment?id=202304050004

In [110]:
exp3_one_shot = pd.read_csv("../pred_files/mt5_lcquad_tokenizer/1_mt5_lcquadtokenizer/results.csv")
exp3_one_shot

,Language,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,Macro F1 QALD
0,ba,0.0032,0.1932,0.0016,0.0585,0.0588,0.0581,0.1089
1,be,0.0086,0.1854,0.0044,0.1107,0.1213,0.1091,0.1892
2,de,0.0082,0.1813,0.0042,0.1005,0.1072,0.0986,0.1710
3,en,0.0091,0.2416,0.0046,0.1666,0.1726,0.1662,0.2718
4,fr,0.0067,0.3608,0.0034,0.0776,0.0806,0.0760,0.1407
5,hy,0.0000,0.0000,0.0000,0.0368,0.0368,0.0368,0.0707
6,ja,0.0038,0.1690,0.0019,0.0886,0.0956,0.0879,0.1569
7,lt,0.0071,0.1728,0.0036,0.0793,0.0861,0.0771,0.1406
8,ru,0.0077,0.2111,0.0039,0.1122,0.1236,0.1088,0.1882
9,uk,0.0085,0.1648,0.0044,0.1034,0.1216,0.1003,0.1749


In [111]:
exp2_one_shot

,Language,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,Macro F1 QALD
0,ba,0.0001,0.0039,0.0000,0.0515,0.0515,0.0515,0.0971
1,be,0.0046,0.1299,0.0023,0.0860,0.0934,0.0854,0.1524
2,de,0.0088,0.1519,0.0046,0.1152,0.1289,0.1112,0.1890
3,en,0.0098,0.2193,0.0050,0.2185,0.2324,0.2143,0.3207
4,fr,0.0064,0.6061,0.0032,0.0650,0.0732,0.0625,0.1175
5,hy,0.0031,0.8727,0.0015,0.0485,0.0512,0.0473,0.0902
6,ja,0.0032,0.0741,0.0016,0.0554,0.0657,0.0521,0.0963
7,lt,0.0063,0.1477,0.0032,0.0503,0.0585,0.0480,0.0903
8,ru,0.0049,0.0936,0.0025,0.1007,0.1086,0.1001,0.1724
9,uk,0.0046,0.0941,0.0023,0.0567,0.0645,0.0621,0.1139


We can see that only `de` and `en` has a lower F1 score than in Experiment 2 one-shot setting. 

### four-shot

Then, we try a four-shot setting, fine-tune with questions in `en`, `de`, `zh`, `ru`

results: https://gerbil-qa.aksw.org/gerbil/experiment?id=202304050001

In [112]:
exp3_four_shot = pd.read_csv("../pred_files/mt5_lcquad_tokenizer/4_mt5_lcquadtokenizer/results.csv")
exp3_four_shot

,Language,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,Macro F1 QALD
0,ba,0.0039,0.2020,0.0020,0.0897,0.0938,0.0887,0.1591
1,be,0.0083,0.1421,0.0043,0.1071,0.1135,0.1054,0.1819
2,de,0.0089,0.1736,0.0046,0.1513,0.1578,0.1496,0.2406
3,en,0.0089,0.1725,0.0046,0.1591,0.1653,0.1642,0.2578
4,fr,0.0067,0.7955,0.0034,0.0775,0.0804,0.0761,0.1411
5,hy,0.0034,0.5300,0.0017,0.0558,0.0584,0.0546,0.1034
6,ja,0.0086,0.2036,0.0044,0.1218,0.1281,0.1201,0.2041
7,lt,0.0086,0.1568,0.0044,0.0925,0.0992,0.0908,0.1595
8,ru,0.0087,0.2094,0.0044,0.1220,0.1356,0.1202,0.2029
9,uk,0.0087,0.1893,0.0044,0.1291,0.1356,0.1274,0.2135


In [113]:
exp2_four_shot

,Language,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,Macro F1 QALD
0,ba,0.0045,0.2023,0.0023,0.1267,0.1342,0.1256,0.2116
1,be,0.0096,0.1835,0.0049,0.1489,0.1593,0.1514,0.2270
2,de,0.0098,0.1708,0.0051,0.2062,0.2177,0.2033,0.2821
3,en,0.0096,0.1340,0.0050,0.1915,0.2030,0.1886,0.2596
4,fr,0.0068,0.5408,0.0034,0.0775,0.0804,0.0761,0.1409
5,hy,0.0037,0.3118,0.0019,0.0579,0.0597,0.0595,0.1119
6,ja,0.0082,0.1904,0.0042,0.1430,0.1526,0.1396,0.2144
7,lt,0.0088,0.1956,0.0045,0.1340,0.1428,0.1311,0.2110
8,ru,0.0095,0.1313,0.0049,0.1548,0.1664,0.1518,0.2236
9,uk,0.0094,0.1421,0.0048,0.1664,0.1766,0.1661,0.2391


As the number of languages in the training set increases, the F1 score for all languages in Experiment 2 four-shot setting is higher than in Experiment 3. This suggests that the language model leverages tokens in the tokenizer when the training dataset is small. However, when the training dataset is large, the language model learns the entity and relation information by itself and the performance is better than using a pre-trained tokenizer.

## Experiment 4

In this experiment, We use mT5-base and the mT5 tokenizer as in the Experiment 2, since this configuration achieves the best performance. The input data consisted of natural language questions with part-of-speech tags, dependency relations, and dependency levels as additional features. The idea of using these features was inspired by [SGPT](https://arxiv.org/abs/2202.08904). We train the model for 300 epochs as in previous experiments. 

In this experiment, we enhance the question string with additional linguistic features, such as part-of-speech tags, dependency relations, and dependency levels, to facilitate the language model's comprehension of the question.

Here, we give some examples of question with linguistic context:

In [114]:
pd.set_option('max_colwidth', 1000)
pd.read_csv("../datasets/11_linguistic/q9pp_train.csv").question.head(3)

0               List all boardgames by GMT . <pad> VERB DET NOUN ADP PROPN PUNCT <pad> ROOT det dobj prep pobj punct <pad> 1 3 2 3 4 2
1     Liste die Brettspiele von GMT auf . <pad> NOUN DET NOUN ADP PROPN ADP PUNCT <pad> ROOT nk oa pg nk svp punct <pad> 1 3 2 3 4 2 2
2    Zeige mir alle Brettspiele von GMT . <pad> VERB PRON DET NOUN ADP PROPN PUNCT <pad> ROOT da nk oa pg nk punct <pad> 1 2 3 2 3 4 2
Name: question, dtype: object

### all languages (11)

We include `en`, `de`, `zh`, `ja`, `ru`, `fr`, `ba`, `be`, `uk`, `lt`, and `es`. 
There is no spacy model for Armenian, therefore Armenian is excluded in this experiment. 

results: https://gerbil-qa.aksw.org/gerbil/experiment?id=202304170000

In [115]:
exp4_all = pd.read_csv("../pred_files/mt5_linguistic/11_linguistic_mt5/results.csv")
exp4_all

,Language,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,Macro F1 QALD
0,ba,0.0066,0.1858,0.0034,0.1590,0.1682,0.1571,0.2498
1,be,0.0101,0.1567,0.0052,0.2133,0.2245,0.2114,0.2851
2,de,0.0101,0.1397,0.0053,0.2253,0.2392,0.2212,0.2939
3,en,0.0103,0.1515,0.0053,0.2432,0.2543,0.2433,0.3200
4,es,0.0100,0.1450,0.0052,0.2353,0.2467,0.2335,0.3161
5,fr,0.0067,0.4380,0.0034,0.0849,0.0878,0.0835,0.1531
6,ja,0.0099,0.1660,0.0051,0.2008,0.2098,0.2047,0.2832
7,lt,0.0095,0.1350,0.0049,0.2179,0.2312,0.2138,0.2988
8,ru,0.0099,0.1348,0.0052,0.2130,0.2243,0.2102,0.2775
9,uk,0.0101,0.1349,0.0052,0.2424,0.2537,0.2396,0.3019


In [116]:
exp2_all

,Language,Micro F1,Micro Precision,Micro Recall,Macro F1,Macro Precision,Macro Recall,Macro F1 QALD
0,ba,0.0056,0.1589,0.0029,0.1571,0.1628,0.1557,0.2457
1,be,0.0098,0.1442,0.0051,0.1986,0.2171,0.1956,0.2704
2,de,0.0099,0.1477,0.0051,0.1962,0.2137,0.1920,0.2634
3,en,0.0099,0.1439,0.0051,0.1914,0.2134,0.1884,0.2585
4,es,0.0100,0.1481,0.0052,0.2059,0.2137,0.2035,0.2787
5,fr,0.0068,0.5354,0.0034,0.0775,0.0804,0.0761,0.1408
6,hy,0.0034,0.1957,0.0017,0.0558,0.0584,0.0546,0.1030
7,ja,0.0104,0.1583,0.0054,0.1966,0.2052,0.1945,0.2625
8,lt,0.0104,0.1307,0.0054,0.1900,0.2089,0.1854,0.2641
9,ru,0.0104,0.1369,0.0054,0.1775,0.1907,0.1742,0.2393


In [ ]:
# TODO: compare with 

According to the results, the majority of languages exhibited an improvement in their F1 score, while some languages still showed comparable F1 scores as those obtained in Experiment 2. Of particular note is the significant improvement of 5.18 in the F1 score for English, which is a notable achievement.